In [1]:
import pandas as pd

In [2]:
df = pd.read_parquet('../do_not_commit/Datasets/Persist_Operating_System_DATA.pq')

In [3]:
features = df.groupby(['MachineID', 'RWB_EFFECTIVE_DATE']).agg(
    {
        'InstallDate00':'max',
        'LastBootUpTime00':'min',
        'TotalVirtualMemorySize00':'max'
    }
).reset_index()
features

,MachineID,RWB_EFFECTIVE_DATE,InstallDate00,LastBootUpTime00,TotalVirtualMemorySize00
0,16783564,2023-01-19,2021-04-02 14:25:05.000,2023-01-16 02:00:46.000,16895
1,16783564,2023-01-20,2021-04-02 14:25:05.000,2023-01-16 02:00:46.000,16895
2,16783564,2023-01-21,2021-04-02 14:25:05.000,2023-01-16 02:00:46.000,16895
3,16783564,2023-01-22,2021-04-02 14:25:05.000,2023-01-16 02:00:46.000,16895
4,16783564,2023-01-23,2021-04-02 14:25:05.000,2023-01-23 02:02:49.000,16895
...,...,...,...,...,...
1041725,16818954,2023-06-28,2022-12-01 12:37:51.000,2023-06-27 10:07:20.000,64963
1041726,16818955,2023-06-28,2023-06-27 11:53:37.000,2023-06-27 12:20:35.000,32490
1041727,16818958,2023-06-28,2022-12-01 07:21:03.000,2022-12-01 07:52:38.000,64963
1041728,16818959,2023-06-28,2023-06-27 14:51:45.000,2023-06-27 15:49:41.000,33573


In [11]:
from datetime import datetime
from datetime import date
import numpy as np

def calculate_age_in_days(target_date_str, in_date):

    # Convert target_date_str to a Python datetime object
    target_date = datetime.strptime(target_date_str, '%Y-%m-%d').date()

    # Calculate the age in days
    age_in_days = (in_date - target_date).days

    return age_in_days


def create_program_features(in_df):

    # Change effective date to type date
    in_df['InstallDate00'] = pd.to_datetime(in_df['InstallDate00']).dt.strftime('%Y-%m-%d')

    # Get age in days
    todaydate = date(2023, 7, 4)
    in_df['InstallAge'] = in_df['InstallDate00'].apply(lambda x: calculate_age_in_days(x, todaydate)
    if pd.notnull(x) else np.nan)
    return in_df


def create_last_boot_features(in_df):

    # Change effective date to type date
    in_df['LastBootUpTime00'] = pd.to_datetime(in_df['LastBootUpTime00']).dt.strftime('%Y-%m-%d')

    # Get age in days
    todaydate = date(2023, 7, 4)
    in_df['LastBootAge'] = in_df['LastBootUpTime00'].apply(lambda x: calculate_age_in_days(x, todaydate)
    if pd.notnull(x) else np.nan)
    return in_df

In [9]:
df = create_program_features(features)

In [12]:
df = create_last_boot_features(df)

In [14]:
df

,MachineID,RWB_EFFECTIVE_DATE,InstallDate00,LastBootUpTime00,TotalVirtualMemorySize00,InstallAge,LastBootAge
0,16783564,2023-01-19,2021-04-02,2023-01-16,16895,823.0,169.0
1,16783564,2023-01-20,2021-04-02,2023-01-16,16895,823.0,169.0
2,16783564,2023-01-21,2021-04-02,2023-01-16,16895,823.0,169.0
3,16783564,2023-01-22,2021-04-02,2023-01-16,16895,823.0,169.0
4,16783564,2023-01-23,2021-04-02,2023-01-23,16895,823.0,162.0
...,...,...,...,...,...,...,...
1041725,16818954,2023-06-28,2022-12-01,2023-06-27,64963,215.0,7.0
1041726,16818955,2023-06-28,2023-06-27,2023-06-27,32490,7.0,7.0
1041727,16818958,2023-06-28,2022-12-01,2022-12-01,64963,215.0,215.0
1041728,16818959,2023-06-28,2023-06-27,2023-06-27,33573,7.0,7.0


In [15]:
features.to_parquet('../do_not_commit/FeatureDatasets/operating_system_features.pq')


# DEPRECATED.



# os_install_date_age


In [4]:
# Age of the operating system calculated from the os_install_age.
# .groupby([MachineID, RWB_effective_date]) and use the .max() value of 'os_install_age'.

# RJ 08/01/23: 'os_install_age' doesn't exist. Using 'InstallDate00'.
# os_install_age = df.groupby(['MachineID', 'RWB_EFFECTIVE_DATE'])['InstallDate00'].agg('max').reset_index()
# os_install_age.to_parquet('../do_not_commit/FeatureDatasets/os_install_age.pq')
# os_install_age


# last_boot_up_time


In [5]:
# Number of days since the machine was last booted.
# Can calculate this from the Operating_System_DATA table using the LastBootUpTime0 attribute.
# .groupby([MachineID, RWB_effective_date]) and use the .min() value of 'LastBootUpTime00'.

# last_boot = df.groupby(['MachineID', 'RWB_EFFECTIVE_DATE'])['LastBootUpTime00'].agg('min').reset_index()
# last_boot.to_parquet('../do_not_commit/FeatureDatasets/last_boot_up_time.pq')
# last_boot


# os_total_virtual_memory_size


In [6]:
# Total virtual operating system memory size.
# .groupby([MachineId, RWB_effective_date]) and use the .max() value of 'TotalVirtualMemorySize00'.

# os_total_virtual_memory_size = df.groupby(['MachineID', 'RWB_EFFECTIVE_DATE'])['TotalVirtualMemorySize00'].agg('max').reset_index()
# os_total_virtual_memory_size.to_parquet('../do_not_commit/FeatureDatasets/os_total_virtual_memory_size.pq')
# os_total_virtual_memory_size